### Imports

### Defining parameters

In [ ]:
# Set the initial conditions
q0 = 1.0       # initial position
p0 = 0.0       # initial momentum

# Set the simulation parameters
dt_leap = 0.1       # stepsize
dt_euler = 0.1
t_max = 10.0   # maximum simulation time
m = 1
K = 1

In [ ]:
def hamiltonian_dynamics(p, q): # for harmonic oscillator (991) and (992) in optimization
    return np.array([p/m, -K*q])

def gradient_potenital_energy(q): 
    return K*q

In [ ]:
# Define the Leapfrog integrator
def leapfrog(q, p, dt):
    p_half = p - 0.5*dt*gradient_potenital_energy(q)
    q_next = q + dt*p_half
    p_next = p_half - 0.5*dt*gradient_potenital_energy(q_next)
    return q_next, p_next

# Initialize the trajectory arrays
t_leapfrog = np.arange(0, t_max, dt_leap)
q_leapfrog = np.zeros_like(t_leapfrog)
p_leapfrog = np.zeros_like(t_leapfrog)

# Set the initial values
q_leapfrog[0] = q0
p_leapfrog[0] = p0

# Run the simulation using the Leapfrog integrator
for i in range(1, len(t_leapfrog)):
    q_leapfrog[i], p_leapfrog[i] = leapfrog(q_leapfrog[i-1], p_leapfrog[i-1], dt_leap)

# Plot the trajectory in phase space
plt.plot(q_leapfrog, p_leapfrog, linewidth=2.5)
plt.xlabel('Position q', fontsize = 14)
plt.ylabel('Momentum p', fontsize = 14)
plt.title('Harmonic oscillator phase space trajectory - Leapfrog', fontsize=14)
plt.show()

#### Explicit Euler for harmonic oscillator